# Week 5: Applied Data Science Capstone Notebook

Now that you have been equipped with the skills and the tools to use location data to explore a geographical location, over the course of two weeks, you will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve. If you cannot think of an idea or a problem, here are some ideas to get you started:

In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Toronto or Paris or some other multicultural city? I will leave it to you to refine this idea.

In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?
These are just a couple of many ideas and problems that can be solved using location data in addition to other datasets. No matter what you decide to do, make sure to provide sufficient justification of why you think what you want to do or solve is important and why would a client or a group of people be interested in your project.

Review criteria

This capstone project will be graded by your peers. This capstone project is worth 70% of your total grade. The project will be completed over the course of 2 weeks. Week 1 submissions will be worth 30% whereas week 2 submissions will be worth 40% of your total grade.

For this week, you will required to submit the following:

* A description of the problem and a discussion of the background. (15 marks)
* A description of the data and how it will be used to solve the problem. (15 marks)

For the second week, the final deliverables of the project will be:

* A link to your Notebook on your Github repository, showing your code. (15 marks)
* A full report consisting of all of the following components (15 marks):
    * Introduction where you discuss the business problem and who would be interested in this project.
    * Data where you describe the data that will be used to solve the problem and the source of the data.
    * Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
    * Results section where you discuss the results.
    * Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
    * Conclusion section where you conclude the report.
* Your choice of a presentation or blogpost. (10 marks)

---------------------------------------------------------

### import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip3 install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip3 install folium==0.5.0 # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### download csv file containing latitudes and longitudes for each postcode

In [3]:
!wget -q -O 'toronto_data.csv' https://cocl.us/Geospatial_data

with open('toronto_data.csv') as toronto_csv:
    toronto_data = pd.read_csv(toronto_csv)

### a few more imports for parsing the Wikipedia page

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
#import html as html_module
from lxml import html as html_module
#from HTMLParser import HTMLParser

### get the Wikipedia page

In [5]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html=urlopen(url)

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
tree = html_module.fromstring(page.content)

soup = BeautifulSoup(html)
df_of_postal_codes = soup.table


### locate the table in the Wikipedia page

In [6]:
neighborhoods_data = soup.find('table',{'class':'wikitable sortable'})


columns = soup.find_all('th')
rows = soup.find_all('td')

rows_array = list()

tbody_contents = soup.tbody.contents
counter = 0
for row in tbody_contents:
    if counter % 2 == 0:
        rows_array.append(row)
    counter += 1

### since the imported table has three empty columns, label them as 'DROP_ME' when creating a DataFrame

In [7]:
column_names = ['DROP_ME', 'Postcode', 'DROP_ME', 'Borough', 'DROP_ME', 'Neighborhood']
column_names_clean = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(data=rows_array, columns=column_names)
    

### drop empty columns from DataFrame

In [8]:
### drop empty columns
df.drop(columns=['DROP_ME'], axis=1, inplace=True)

### parse the text for each row by removing the leading newline character and replacing the rest with commas, then create a list of the rows

In [9]:
data = [element.text.strip("\n").replace("\n", ",") for element in soup.find_all("tr", limit=288)]

### create a new list containing the cleaned data, which is split using the commas inserted in the previous step

In [10]:
data_clean = list()
for row in data:
    data_clean.append(row.split(","))

### create a DataFrame from the cleaned data

In [11]:
df_clean = pd.DataFrame(data=data_clean[1:][:], columns=data_clean[0][0:3])

### don't process rows with no assigned borough

In [12]:
counter = 0
for row in df_clean['Borough']:
    if row == 'Not assigned':
        df_clean.drop(index=counter, inplace=True)
    counter += 1

### replace unassigned neighborhoods with the names of their boroughs

In [13]:
counter = 0
for row in df_clean['Neighbourhood']:
    if row == 'Not assigned':
        df_clean['Neighbourhood'].iloc[counter] = df_clean['Borough'].iloc[counter]
    counter += 1
    

### combine neighborhoods for rows with duplicate postcodes

In [14]:
counter = 0
for row in df_clean['Postcode']:
    #print(counter, len(df_clean['Postcode']), row)
    if counter < len(df_clean['Postcode']) - 1 and row == df_clean['Postcode'].iloc[counter+1]:
        #print('duplicate postcodes found', row)
        df_clean['Neighbourhood'].iloc[counter+1] += ", " + df_clean['Neighbourhood'].iloc[counter]
    counter += 1
    

### keep only the last of each postcode's duplicates, as that entry contains all the neighborhoods

In [15]:
df_clean.drop_duplicates(subset='Postcode', keep='last', inplace=True)

### add latitude and longitude columns to DataFrame from Geospatial_Coordinates.csv

In [16]:
# instantiate the dataframe
df_coords = pd.read_csv('Geospatial_Coordinates.csv')
df_clean.sort_values(axis=0, by=['Postcode'], inplace=True)
df_clean = df_clean.reset_index(drop=True)
df_toronto = df_clean.join(df_coords)
df_toronto.drop(columns='Postal Code', inplace=True)
display(df_toronto)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


### get the geographical coordinates of Toronto

In [154]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### define Foursquare Credentials and Version

In [155]:
CLIENT_ID = 'K4LLRH2S53DQO5LZOAEFYYNJQHD0HFMR0MQRGJSSLCEV3P0T' # your Foursquare ID
CLIENT_SECRET = 'WNBDPYS4MRKVBTDQMEMTO4AK4NBRLEAMHARD5JZTDJREEJIW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K4LLRH2S53DQO5LZOAEFYYNJQHD0HFMR0MQRGJSSLCEV3P0T
CLIENT_SECRET:WNBDPYS4MRKVBTDQMEMTO4AK4NBRLEAMHARD5JZTDJREEJIW


Get the neighborhood's name.

In [156]:
toronto_data.loc[0]['Neighbourhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.

In [157]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


### get the top 1000 venues that are within a radius of 1 kilometer

First, let's create the GET request URL. Name your URL **url**.

In [158]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=K4LLRH2S53DQO5LZOAEFYYNJQHD0HFMR0MQRGJSSLCEV3P0T&client_secret=WNBDPYS4MRKVBTDQMEMTO4AK4NBRLEAMHARD5JZTDJREEJIW&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=1000'

### send the GET request and examine the resutls

In [159]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dfe7ca292393563120645b9'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs'

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

### define function to get venue categories

In [160]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

### clean the results JSON and convert it to a DataFrame

In [161]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy's,Fast Food Restaurant,43.807448,-79.199056
3,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777


And how many venues were returned by Foursquare?

In [162]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


#### create a function to repeat the same process to all the neighborhoods in Toronto

In [163]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### create DataFrame containing venues near each neighborhood

In [164]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Malvern, Rouge
Port Union, Rouge Hill, Highland Creek
West Hill, Morningside, Guildwood
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Oakridge, Golden Mile, Clairlea
Scarborough Village West, Cliffside, Cliffcrest
Cliffside West, Birch Cliff
Wexford Heights, Scarborough Town Centre, Dorset Park
Wexford, Maryvale
Agincourt
Tam O'Shanter, Sullivan, Clarks Corners
Steeles East, Milliken, L'Amoreaux East, Agincourt North
L'Amoreaux West
Upper Rouge
Hillcrest Village
Oriole, Henry Farm, Fairview
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South, Flemingdon Park
Wilson Heights, Downsview North, Bathurst Manor
York University, Northwood Park
Downsview East, CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
Riverdale, The Danf

### check the size of the resulting dataframe

In [141]:
print(toronto_venues.shape)
toronto_venues.head()

(2213, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"West Hill, Morningside, Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"West Hill, Morningside, Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"West Hill, Morningside, Guildwood",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


### check how many venues were returned for each neighborhood

In [142]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
Caledonia-Fairbanks,5,5,5,5,5,5
Canada Post Gateway Processing Centre,10,10,10,10,10,10
Cedarbrae,8,8,8,8,8,8
Central Bay Street,85,85,85,85,85,85
Christie,17,17,17,17,17,17


### find out how many unique categories can be curated from all the returned venues

In [143]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 unique categories.


### analyze each neighborhood via one-hot encoding of the venue categories

In [144]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### examine the new dataframe size

In [145]:
toronto_onehot.shape

(2213, 273)

### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [146]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

### confirm the new size

In [147]:
toronto_grouped.shape

(100, 273)

### print each neighborhood along with the top 5 most common venues

In [148]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3             Sandwich Place  0.25
4             Medical Center  0.00


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1      Steakhouse  0.04
2     Cheese Shop  0.04
3    Cocktail Bar  0.04
4  Farmers Market  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3                 Spa  0.06
4          Smoke Shop  0.06


----Caledonia-Fairbanks----
                  venue  freq
0                  Park   0.4
1         Women's Store   0.2
2                Market   0.2
3  Fast Food Restaurant   0.

#### Let's put that into a *pandas* dataframe

### write a function to sort the venues in descending order

In [149]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### create the new dataframe and display the top 25 venues for each neighborhood

In [150]:
num_top_venues = 25

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Sandwich Place,Breakfast Spot,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Drugstore,Dessert Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Dim Sum Restaurant,Department Store,Event Space,Convenience Store,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store,Dessert Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Dim Sum Restaurant,Deli / Bodega,Department Store,Falafel Restaurant,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant
2,Berczy Park,Coffee Shop,Beer Bar,Café,Bakery,Steakhouse,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Sandwich Place,Concert Hall,Shopping Mall,Bistro,Irish Pub,Hotel,Beach,Breakfast Spot,Japanese Restaurant,Restaurant,Sporting Goods Shop,Jazz Club,Butcher,Diner,Liquor Store,Park
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop,Skate Park,Park,Garden Center,Gym / Fitness Center,Pizza Place,Comic Shop,Construction & Landscaping,Concert Hall,College Stadium,Dumpling Restaurant,Drugstore,Colombian Restaurant,Donut Shop,Doner Restaurant
4,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Drugstore,Department Store,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Dessert Shop,Deli / Bodega,Event Space,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space


### filter the venues to only include those for which the category includes the word 'restaurant'

In [151]:
restaurant_counter = 0
list_of_restaurant_cols = ['Neighborhood']

for category in toronto_grouped.columns:
    if category.find('Restaurant') > -1:
        restaurant_counter += 1
        list_of_restaurant_cols.append(category)
        
print("there area a total of {} restaurant categories".format(restaurant_counter))
#display(toronto_grouped[list_of_restaurant_cols])

toronto_restaurants = toronto_onehot.groupby('Neighborhood')
#display(toronto_restaurants)

# one hot encoding
#toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#toronto_restaurants['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_restaurants.columns[-1]] + list(toronto_restaurants.columns[:-1])
#toronto_restaurants = toronto_restaurants[fixed_columns]

toronto_restaurants = toronto_restaurants[list_of_restaurant_cols]

#if toronto_venues['Venue Category'] in list_of_restaurant_cols:
#    print('found one')

there area a total of 51 restaurant categories


### since we're searching for restaurants within the same radius for each row, each row's restaurant density can be simply computed by computing the area of the search circle

In [166]:
search_area = radius * radius * np.pi
restaurant_density = lambda x: x / search_area

#toronto_restaurants.groupby('Neighborhood')
toronto_restaurants_count   = toronto_restaurants.count().iloc[:,0].copy()

#for row in toronto_restaurants_count:
#    print(row)
toronto_restaurants_density = toronto_restaurants_count / search_area

display(toronto_restaurants_density.sort_values())
#display(toronto_restaurants_count.sort_values())

Neighborhood
Port Union, Rouge Hill, Highland Creek                                                                                                    3.183099e-07
Malvern, Rouge                                                                                                                            3.183099e-07
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale                                                                    3.183099e-07
Weston                                                                                                                                    3.183099e-07
Scarborough Village                                                                                                                       3.183099e-07
Humberlea, Emery                                                                                                                          3.183099e-07
York Mills, Silver Hills                                                         

## Based on these restaurant densities, computed for circles of radius 1 km around the center of each group of neighborhoods, the new restaurant will have the least competition in any of the following neighborhoods:

* Port Union, Rouge Hill, Highland Creek
* Malvern, Rouge
* West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
* Weston
* Scarborough Village
* Humberlea, Emery
* York Mills, Silver Hills

## At the other end of the spectrum, the new restaurant will have the most competition if it opens in any of the following neighborhoods:

* Toronto Dominion Centre, Design Exchange
* St. James Town
* Victoria Hotel, Commerce Court
* Garden District, Ryerson
* Union Station, Toronto Islands, Harbourfront East
* Underground city, First Canadian Place
* Richmond, King, Adelaide